In [ ]:
%matplotlib inline

In [ ]:
import sys
import os 
from glob import glob
import json

In [ ]:
sys.path.insert(0, '../../')

In [ ]:
import paleopy

In [ ]:
from paleopy import plotting

In [ ]:
plotting.scalar_plot

In [ ]:
from paleopy import Proxy 
from paleopy import Analogs
from paleopy import Ensemble
from paleopy.plotting import scalar_plot

In [ ]:
djsons = '../../jsons/'
pjsons = '../../jsons/proxies'

In [ ]:
from numpy import ma
import numpy as np
import xarray as xray

In [ ]:
# !ls ../../jsons/proxies/

In [ ]:
# ljson = glob('../../jsons/proxies/*.json')

### modify ALL the proxies so that they include the weights

In [ ]:
# for file_in in ljson: 
#     file_out = file_in[:-5] + '_proxy.json'
#     with open(file_in, 'r') as fin: 
#         with open(file_out,'w') as fout:
#             proxy = json.load(fin)
#             proxy['weights'] = np.ones(len(proxy['analog_years'])).tolist()
#             json.dump(proxy, fout)

### test the new implementation of PALEOPY with just one proxy

In [ ]:
p = Proxy(sitename='Rarotonga', \
          lon = -159.82, \
          lat = -21.23, \
          djsons = djsons, \
          pjsons = pjsons, \
          pfname = 'Rarotonga.json', \
          dataset = 'ersst', \
          variable ='sst', \
          measurement ='delta O18', \
          dating_convention = 'absolute', \
          calendar = 'gregorian',\
          chronology = 'historic', \
          season = 'DJF', \
          value = 0.6, \
          qualitative = 0, \
          calc_anoms = True, \
          detrend = True, \
        method = 'quintiles')

In [ ]:
p.find_analogs()

In [ ]:
p.proxy_repr(pprint=True, outfile=False)

In [ ]:
p.analog_years

In [ ]:
p.season

In [ ]:
p.detrend

In [ ]:
f = p.plot_season_ts()

In [ ]:
a = Analogs(p, 'ncep', 'hgt_1000')

In [ ]:
a

In [ ]:
hgt_1000 = a.composite()

In [ ]:
scalar_plot(hgt_1000,test=0.05).plot()

### example with a ensemble of consistent proxies 

In [ ]:
ens = Ensemble(djsons=djsons, pjsons=pjsons)

In [ ]:
len(ens.df_proxies)

In [ ]:
b = Analogs(ens, 'ncep', 'hgt_1000').composite()

In [ ]:
f = scalar_plot(b,test=0.05).plot()

In [ ]:
b.dset_compos['composite_anomalies'].plot()

In [ ]:
len(ens.df_proxies)

In [ ]:
len(ens.analog_years)

In [ ]:
ens.df_proxies.head()

### test now with an INCONSISTENT ensemble 

In [ ]:
ens = Ensemble(djsons=djsons, pjsons='../../jsons/proxies/inconsistents/')

In [ ]:
ens.df_proxies.head()

In [ ]:
ens.detrend

In [ ]:
ens.climatology

In [ ]:
c = Analogs(ens, 'ncep', 'hgt_1000')

In [ ]:
c

In [ ]:
c.parent.description

In [ ]:
c.composite()

In [ ]:
c.dset_compos

In [ ]:
from sys import getsizeof

In [ ]:
getsizeof(c.dset_compos)

In [ ]:
c.dset_compos

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
c.dset_compos['composite_anomalies'].plot(x='longitudes', y='latitudes', col='proxy', col_wrap=5)
f = plt.gcf()
f.savefig('/Users/nicolasf/Desktop/dummy_ensemble_plot.png', dpi=200)

In [ ]:
a

In [ ]:
b

In [ ]:
c

In [ ]:
hgt850 = Analogs(p, 'ncep', 'hgt_850').composite()
uwnd850 = Analogs(p, 'ncep', 'uwnd_850').composite()
vwnd850 = Analogs(p, 'ncep', 'vwnd_850').composite()

In [ ]:
hgt850.dset_compos.composite_anomalies.plot()

In [ ]:
uwnd850.dset_compos.composite_anomalies.plot()

In [ ]:
vwnd850.dset_compos.composite_anomalies.plot()

In [ ]:
vwnd850

In [ ]:
vwnd850.locations

In [ ]:
uanoms = uwnd850.dset_compos['composite_anomalies']

In [ ]:
vanoms = vwnd850.dset_compos['composite_anomalies']

In [ ]:
windspeed = np.sqrt(uanoms**2 + vanoms**2)

In [ ]:
windspeed.plot()

In [ ]:
from paleopy.plotting import vector_plot

In [ ]:
uwnd850.dset_compos

In [ ]:
c = vector_plot(uwnd850, vwnd850, hgt850)